In [1]:
import spacy

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
texts = [
    "Net income was $9.4 million compared to the prior year of $2.7 million.",
    "Revenue exceeded twelve billion dollars, with a loss of $1b.",
]

In [4]:
for doc in nlp.pipe(texts, disable=["tagger", "parser"]):
    # Do something with the doc here
    print([(ent.text, ent.label_) for ent in doc.ents])

[W108] The rule-based lemmatizer did not find POS annotation for the token 'Net'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'income'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'was'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token '$'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token '9.4'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'a

[('$9.4 million', 'MONEY'), ('the prior year', 'DATE'), ('$2.7 million', 'MONEY')]
[('twelve billion dollars', 'MONEY'), ('1b', 'MONEY')]


In [5]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x121466720>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x12174b900>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1217143a0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1217144c0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1217cf780>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1217a1780>)]

In [6]:
from spacy.lang.en import English

In [7]:
nlp = English()

In [9]:
doc = nlp("This is a sentence")

In [13]:
doc.

This
is
a
sentence
